In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
from numpy.linalg import inv
import numpy.matlib
import warnings
warnings.filterwarnings('ignore')

# This forward problem describes the straight-line ray path travel time
# from a source with an unknown origin time to a receiver in a
# homogeneous medium with unknown velocity V.
#
# NOTE: variables such as minitial, mtarget, and eobs may be over-written
# within optimization_method.ipynb when the user specifies multiple runs.
#
# CALLED BY : hw_genlsq.ipynb
#
# contributors: Carl Tape, Nealey Sims*, Amanda McPherson*
#
### Must be run within hw_genlsq.ipynb in order to obtain key variables ###

In [ ]:
run_forward_epicenter=False      #default=False; run without genlsq = True

### be sure to change again to avoid changing variables in multiple locations

if run_forward_epicenter==True:
    # USER INPUT
    nsamples = 1000
    irandom_initial_model = 0      # 0(fixed), 1(random)
    irandom_target_model = 0       # 0(fixed), 1(random)
    idata_errors = 2               # 0(none),  1(random), 2(fixed)
    ifig = 1                       # 0,1
    
    inormalization = 1
    stnsamples = str(nsamples) + ' samples'
    stlabS = ('Sd(m^n)','Sm(m^n)','S(m^n) = Sd + Sm')

In [ ]:
# FORWARD PROBLEM 

ndata = 12
nparm = 4

# labels for parameters
mlabs = ['xs','ys','ts','v']
ulabs = ['km','km','s','none']

# velocity for prior model
# v is the unitless logarithmic velocity, v = ln(V/V0)
# V = V0*exp(v) has units km/s
V = 5      # km/s
V0 = 1     # arbitrary scale factor (V is defined below)

# travel time computation (homogeneous velocity; straight ray paths)

def d2(x,y,xr,yr):
    d2x = (xr-x)**2 + (yr-y)**2
    return d2x
def d1(x,y,xr,yr):
    d1x = np.sqrt(d2(x,y,xr,yr))
    return d1x
def tt(x,y,ts,v,xr,yr):
    ttx = ts + (d1(x,y,xr,yr)/(V0*math.exp(v)))
    return ttx

# receiver locations for uniform grid
xrecmin = 10
xrecmax = 80
yrecmin = 20
yrecmax = 90
xvec  = np.linspace(xrecmin,xrecmax,4)
yvec  = np.linspace(yrecmin,yrecmax,3)
[X,Y] = np.meshgrid(xvec,yvec)
xrec  = X.flatten(order='F')
yrec  = Y.flatten(order='F')

# computation of predictions vector d = g(m)
def d(m):
    dm = np.array([tt(m[0],m[1],m[2],m[3],xrec[0],yrec[0]),
                   tt(m[0],m[1],m[2],m[3],xrec[1],yrec[1]),
                   tt(m[0],m[1],m[2],m[3],xrec[2],yrec[2]),
                   tt(m[0],m[1],m[2],m[3],xrec[3],yrec[3]),
                   tt(m[0],m[1],m[2],m[3],xrec[4],yrec[4]),
                   tt(m[0],m[1],m[2],m[3],xrec[5],yrec[5]),
                   tt(m[0],m[1],m[2],m[3],xrec[6],yrec[6]),
                   tt(m[0],m[1],m[2],m[3],xrec[7],yrec[7]),
                   tt(m[0],m[1],m[2],m[3],xrec[8],yrec[8]),
                   tt(m[0],m[1],m[2],m[3],xrec[9],yrec[9]),
                   tt(m[0],m[1],m[2],m[3],xrec[10],yrec[10]),
                   tt(m[0],m[1],m[2],m[3],xrec[11],yrec[11])],dtype='float')
    return dm

# ndata x nparm matrix of partial derivatives (differentiate tt with respect to each parameter)
# evaluated at model m = (xs,ys,ts,v)

def G(m):
    Gm = np.array([[-(d2(m[0],m[1],xrec[0],yrec[0]))**(-1/2)@(xrec[0]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[0],yrec[0]))**(-1/2)@(yrec[0]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[0],yrec[0])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[1],yrec[1]))**(-1/2)@(xrec[1]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[1],yrec[1]))**(-1/2)@(yrec[1]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[1],yrec[1])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[2],yrec[2]))**(-1/2)@(xrec[2]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[2],yrec[2]))**(-1/2)@(yrec[2]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[2],yrec[2])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[3],yrec[3]))**(-1/2)@(xrec[3]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[3],yrec[3]))**(-1/2)@(yrec[3]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[3],yrec[3])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[4],yrec[4]))**(-1/2)@(xrec[4]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[4],yrec[4]))**(-1/2)@(yrec[4]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[4],yrec[4])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[5],yrec[5]))**(-1/2)@(xrec[5]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[5],yrec[5]))**(-1/2)@(yrec[5]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[5],yrec[5])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[6],yrec[6]))**(-1/2)@(xrec[6]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[6],yrec[6]))**(-1/2)@(yrec[6]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[6],yrec[6])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[7],yrec[7]))**(-1/2)@(xrec[7]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[7],yrec[7]))**(-1/2)@(yrec[7]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[7],yrec[7])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[8],yrec[8]))**(-1/2)@(xrec[8]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[8],yrec[8]))**(-1/2)@(yrec[8]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[8],yrec[8])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[9],yrec[9]))**(-1/2)@(xrec[9]-m[0])/(V0*math.exp(m[3]))    ,  -(d2(m[0],m[1],xrec[9],yrec[9]))**(-1/2)@(yrec[9]-m[1])/(V0*math.exp(m[3]))    ,  1 , -d1(m[0],m[1],xrec[9],yrec[9])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[10],yrec[10]))**(-1/2)@(xrec[10]-m[0])/(V0*math.exp(m[3])) ,  -(d2(m[0],m[1],xrec[10],yrec[10]))**(-1/2)@(yrec[10]-m[1])/(V0*math.exp(m[3])) ,  1 , -d1(m[0],m[1],xrec[10],yrec[10])/(V0*math.exp(m[3]))],
                   [-(d2(m[0],m[1],xrec[11],yrec[11]))**(-1/2)@(xrec[11]-m[0])/(V0*math.exp(m[3])) ,  -(d2(m[0],m[1],xrec[11],yrec[11]))**(-1/2)@(yrec[11]-m[1])/(V0*math.exp(m[3])) ,  1 , -d1(m[0],m[1],xrec[11],yrec[11])/(V0*math.exp(m[3]))]
    ],dtype='float')
    return Gm

# nparm x nparm matrix of second partial derivatives (only used in full Newton method)
# note: this contains the measurement index ii

def G2(m,ii):
    G2m = np.array([
    [(d1(m[0],m[1],xrec[ii],yrec[ii]))**-3@(yrec[ii]-m[1])**2/(V0*math.exp(m[3])),                  -(d1(m[0],m[1],xrec[ii],yrec[ii]))**-3@(xrec[ii]-m[0])*(yrec[ii]-m[1])[0]/(V0*math.exp(m[3])), 0, (d1(m[0],m[1],xrec[ii],yrec[ii]))**-1@(xrec[ii]-m[0])/(V0*math.exp(m[3]))],
    [-(d1(m[0],m[1],xrec[ii],yrec[ii]))**-3@(xrec[ii]-m[0])*(yrec[ii]-m[1])[0]/(V0*math.exp(m[3])),  (d1(m[0],m[1],xrec[ii],yrec[ii]))**-3@(xrec[ii]-m[0])**2/(V0*math.exp(m[3])),                 0, (d1(m[0],m[1],xrec[ii],yrec[ii]))**-1@(yrec[ii]-m[1])/(V0*math.exp(m[3]))],
    [0,                                                                                              0,                                                                                            0, 0],
    [(d1(m[0],m[1],xrec[ii],yrec[ii]))**-1@(xrec[ii]-m[0])/(V0*math.exp(m[3])),                      (d1(m[0],m[1],xrec[ii],yrec[ii]))**-1@(yrec[ii]-m[1])/(V0*math.exp(m[3])),                    0, d1(m[0],m[1],xrec[ii],yrec[ii])[0]/(V0*math.exp(m[3]))]])
    return G2m

In [ ]:
# RANDOM VECTORS (for sampling covariance matrices)

# Gaussian random vectors, each with mean = 0 and standard deviation = 1
randn_vecs_m = np.random.randn(nparm,nsamples)   # model
randn_vecs_d = np.random.randn(ndata,nsamples)   # data

In [ ]:
# PRIOR MODEL (MEAN MODEL) : xs, ys, ts, v

# prior model
# note: V and V0 are defined above
mprior = np.array([[35, 45, 16, np.log(V/V0)]]).T

# prior model covariance matrix (assumed to be diagonal)
sigma_prior = np.array([10, 10, 0.5, 0.2])        # standard deviations
cprior0     = np.diag( np.square(sigma_prior) )   # diagonal covariance matrix
if inormalization==1:
    Cmfac = nparm
else:
    Cmfac = 1

cprior   = Cmfac*cprior0                # with normalization factor
icprior  = inv(cprior)                  # with normalization factor
icprior0 = inv(cprior0)                 # without normalization factor
Lprior   = np.linalg.cholesky(cprior0)  # square-root (lower triangular)
# sample the prior model distribution using the square-root UNNORMALIZED covariance matrix
#cov_samples_m  = np.dot(Lprior, randn_vecs_m)
cov_samples_m  = Lprior @ randn_vecs_m
mprior_samples = np.matlib.repmat(mprior,1,nsamples) + cov_samples_m

# compute the norm of each model sample using the inverse NORMALIZED covariance matrix
norm2_mprior = np.zeros((nsamples,1))
for xx in range(nsamples):
    dm = np.array([mprior_samples[:,xx]]).T - mprior
    norm2_mprior[xx] = dm.T @ icprior @ dm
#figure; plot(norm2_mprior,'.')

In [ ]:
# INITIAL MODEL

# minitial is DIFFERENT FOR EACH RUN, or you can fix it for testing purposes
if irandom_initial_model == 1:
    minitial = np.array([mprior_samples[:,0]]).T   # first sample (random)
else:
    minitial = np.array([                          # fixed
        [46.5236],
        [40.1182],
        [15.3890],
        [1.7748]])

In [ ]:
# TARGET MODEL

# mtarget is DIFFERENT FOR EACH RUN, or you can fix it for testing purposes
if irandom_target_model == 1 : 
    mtarget = np.array([mprior_samples[:,-1]]).T   # last sample (random)
else:
    mtarget = np.array([                           # fixed  
           [21.2922],
           [46.2974],
           [16.1314],
            [2.0903]])

In [ ]:
# TARGET DATA

dtarget = np.array(d(mtarget))
# data covariance matrix (assumed to be diagonal)
tsigma = 0.5                                 # uncertainty in arrival time measurement, seconds
sigma_obs = tsigma * np.ones((ndata))        # standard deviations
cobs0     = np.diag( np.square(sigma_obs) )  # diagonal covariance matrix

if inormalization==1:
    Cdfac = ndata
else:
    Cdfac = 1

cobs   = Cdfac * cobs0              # with normalization factor
icobs  = inv(cobs)                  # with normalization factor
icobs0 = inv(cobs0)                 # without normalization factor
Lcobs  = np.linalg.cholesky(cobs0)  # square-root (lower triangular)

# sample the data distribution using the square-root UNNORMALIZED covariance matrix
cov_samples_d = Lcobs @ randn_vecs_d
dobs_samples  = np.matlib.repmat(dtarget,1,nsamples) + cov_samples_d

# compute the norm of each data sample using the inverse NORMALIZED covariance matrix
norm2_dobs = np.zeros((nsamples,1))
for xx in range(nsamples):
    dd = np.array([dobs_samples[:,xx]]).T - dtarget
    norm2_dobs[xx] = dd.T @ icobs @ dd

# Pick the uncertainties for the target data by simply choosing a sample
#   from the realizations of the data covariance.
# NOTE: eobs is DIFFERENT FOR EACH RUN, or you can fix it for testing purposes

if idata_errors == 0:
    eobs = np.zeros((ndata,1))               # no errors
elif idata_errors == 1:
    eobs = np.array([cov_samples_d[:,1]]).T  # first sample (random)
elif idata_errors == 2:
    eobs = np.array([[                       # fixed
            -0.8689,
            -0.4666,
            -0.0516,
            0.2411,
            0.2825,
            -0.2301,
            0.1977,
            0.3291,
            1.0063,
            0.5674,
            0.1348,
            0.4603]]).T

# "true" observations (includes added errors)
dobs = dtarget + eobs

In [ ]:
def plot_epicenters(mprior_samples,mprior,minitial,mtarget,opts,mpost=[]):
    #PLOT_EPICENTERS plots epicenters within the source-station geometry
    #
    # Carl Tape, 2010-02-24
    
    # input options
    xrec = opts[0]
    yrec = opts[1]
    iray = opts[2]
    ax0  = opts[3]
    
    nparm = np.shape(mprior_samples)[0]
    nsample = np.shape(mprior_samples)[1]
    ndata = len(xrec)
    
    # indices of xs and ys within model vector
    ix = 0
    iy = 1
    
    fig = plt.figure(figsize=(8,8))
    #plt.subplot(aspect=1)
    msizer = 20    # receiver size
    msizes = 15    # source size
    rthick = 1     # receiver edge thickness
    sthick = 2     # source edge thickness
    rfsize = 10
    
    #xmin = ax0(1);
    #xmax = ax0(2);
    #ymin = ax0(3);
    #ymax = ax0(4);
    
    # plot ray paths
    if iray==1:
        for ii in range(ndata):
            plt.plot([minitial[ix], xrec[ii]],[minitial[iy], yrec[ii]],'k')
            
    # plot option depends on if a posterior model is passed
    if len(mpost)>0:
        p0 = plt.plot(mprior_samples[ix,:],mprior_samples[iy,:],'b.')
        p1 = plt.plot(minitial[ix],minitial[iy],'o')
        p2 = plt.plot(mpost[ix],mpost[iy],'co',markersize=msizes,markeredgecolor='w',markeredgewidth='3',linewidth=sthick)
        pP = plt.plot(mprior[ix],mprior[iy],'bo',markersize=msizes,markeredgecolor='w',markeredgewidth='3',linewidth=sthick)
        pT = plt.plot(mtarget[ix],mtarget[iy],'ro',markersize=msizes,markeredgecolor='w',markeredgewidth='3',linewidth=sthick)
        plt.plot(xrec,yrec,'wv',markersize=msizer,markeredgecolor='k',linewidth=rthick)
        for ii in range(ndata):
            plt.text(xrec[ii],yrec[ii], str(ii+1),fontsize=rfsize,color='r',
                     horizontalalignment='center',verticalalignment='center')

        plt.legend([p0[0], p1[0], p2[0], pP[0], pT[0]],['Cprior sample','minitial','mpost','mprior','mtarget'],loc='upper right')
        
    else:
        if len(mprior_samples)>1:
            p0 = plt.plot(mprior_samples[ix,:],mprior_samples[iy,:],'b.')
        
        p1 = plt.plot(minitial[ix],minitial[iy],'ko',markersize=msizes,markeredgecolor='w',markeredgewidth='3',linewidth=sthick)
        pP = plt.plot(mprior[ix],mprior[iy],'bo',markersize=msizes,markeredgecolor='w',markeredgewidth='3',linewidth=sthick)
        pT = plt.plot(mtarget[ix],mtarget[iy],'ro',markersize=msizes,markeredgecolor='w',markeredgewidth='3',linewidth=sthick)
        plt.plot(xrec,yrec,'wv',markersize=msizer,markeredgecolor='k',linewidth=rthick)
        for ii in range(ndata):
            plt.text(xrec[ii],yrec[ii],str(ii+1),fontsize=rfsize,color='r',
                     horizontalalignment='center',verticalalignment='center')
        if len(mprior_samples)>1:
            plt.legend([p0[0], p1[0], pP[0], pT[0]],['Cprior sample','minitial','mprior','mtarget'],loc='upper right')
            x = mprior_samples[ix,:]
            y = mprior_samples[iy,:]
            #iin = find( and( and( x >= ax0(1), x <= ax0(2)), and(y >= ax0(3), y <= ax0(4))) );
            iin=0
            for jj in range(len(x)):
                if x[jj] >= ax0[0] and x[jj] <=ax0[1] and y[jj] >= ax0[2] and y[jj]<=ax0[3]:
                    iin+=1
            plt.title('%i/%i (%.2f) prior samples inside the region shown' %
                (iin,nsample,iin/nsample))
        else:
            plt.legend([p1[0], pP[0], pT[0]],['minitial','mprior','mtarget'],loc='upper right')
    
    #set(gca,'xtick',[0:20:100],'ytick',[0:20:100]);
    #plt.axis(axepi)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.xlabel('X distance (km)')
    plt.ylabel('Y distance (km)')

In [ ]:
# PLOTS

# check for samples that fall outside these limits
axepi = [0,100,0,100]

# source-receiver geometry, prior samples
# this highlights that minitial and mtarget are in the same cloud
plot_epicenters(mprior_samples,mprior,minitial,mtarget,[xrec,yrec,0,axepi])

# source-receiver geometry, ray paths
# this highlights the evaluation of the misfit function for minitial
plot_epicenters([[]],mprior,minitial,mtarget,[xrec,yrec,1,axepi])

# source-receiver geometry, ray paths, prior samples
#plot_epicenters(mprior_samples,mprior,minitial,mtarget,[xrec,yrec,1,axepi])